[View in Colaboratory](https://colab.research.google.com/github/YogeshSingla/sdp_cnn_implementation/blob/master/sdn_cnn_implementation.ipynb)

In [9]:
import keras
import javalang
print("Cell run")
tree = javalang.parse.parse("package javalang.brewtab.com; class Test {}")
print(tree.package.name)

Cell run
javalang.brewtab.com


## RNN implementation
*Reference: Chapter 14: Recurrent Neural Networks, O'Reilly Hands on Machine Learning with Scikit-Learn and Tensorflow*

In [0]:
# Simple RNN - Unrolled in time
import numpy as np
import tensorflow as tf

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs]) 
"""
note:
none means dimension is not specified. So, it X0 takes any number of rows but
n_inputs number of columns.
"""
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0,Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0,Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

with tf.Session() as sess:
  init.run()
  Y0_val,Y1_val = sess.run([Y0, Y1],feed_dict={X0: X0_batch, X1: X1_batch})


In [8]:
print(Y0_val)
print(X0_batch)

[[-0.96996015 -0.6410698  -0.9988856   0.999174    0.30030066]
 [-1.         -0.98527116 -1.          0.9999995   0.59237576]
 [-1.         -0.9994968  -1.          1.          0.78288746]
 [-1.         -0.86512524 -1.         -1.          0.9133607 ]]
[[0 1 2]
 [3 4 5]
 [6 7 8]
 [9 0 1]]


In [0]:
# RNN using static_rnn function in tf - Unrolled in time 
# note: terminate session and run again if ValueError arises due to variables
#already in kernel (quickfix)
import numpy as np
import tensorflow as tf

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs]) 
"""
note:
none means dimension is not specified. So, it X0 takes any number of rows but
n_inputs number of columns.
"""
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32))

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)
Y0 , Y1 = output_seqs

init = tf.global_variables_initializer()

X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

with tf.Session() as sess:
  init.run()
  Y0_val,Y1_val = sess.run([Y0, Y1],feed_dict={X0: X0_batch, X1: X1_batch})
  

In [2]:
print(Y0_val)

[[-0.5649363   0.9316831  -0.47059372  0.5860515   0.9365775 ]
 [-0.9906842   0.9999971  -0.9396638   0.97385293  0.99985975]
 [-0.99984246  1.         -0.9946387   0.9986562   0.9999997 ]
 [-0.9577375   0.999996    0.6915387  -0.6818011   0.53698236]]


In [1]:
# RNN with sequence input (given in single list) - Unrolled in time
# note: terminate session and run again if ValueError arises due to variables
#already in kernel (quickfix)
import numpy as np
import tensorflow as tf

n_inputs = 3
n_neurons = 5

"""
note:
none means dimension is not specified. So, it X0 takes any number of rows but
n_inputs number of columns.
"""
n_steps = 2
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))
"""
perm is useful to decide the tranformation to be done.
initial is 0,1,2
transposed is 1,0,2 ie only rows and columns are altered. third dimension is kept same.
it is NOT transposed.
in 2D (0,1) is transposed to (1,0)
in 3D by default (0,1,2) is transposed to (2,1,0)
"""

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

init = tf.global_variables_initializer()

X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
    ])

with tf.Session() as sess:
  init.run()
  outputs_val = outputs.eval(feed_dict={X: X_batch})

  
print(outputs_val)

[[[-0.26703614  0.85850936 -0.94624954 -0.89803565  0.90369356]
  [-0.9973372   0.9994185  -0.9999812  -1.          0.9981015 ]]

 [[-0.89537394  0.9956316  -0.99984586 -0.99987924  0.9983182 ]
  [ 0.31190956  0.1610087   0.83435875 -0.9639556  -0.91322875]]

 [[-0.9895204   0.9998741  -0.9999995  -0.99999976  0.99997205]
  [-0.93418723  0.98498064 -0.99407107 -0.9999992   0.76789415]]

 [[-0.99964917 -0.15638609  0.80194604 -0.9255959  -0.9442658 ]
  [-0.4826311   0.61576605 -0.8357857  -0.96386933  0.7421594 ]]]


In [1]:
# RNN with sequence input (given in single list) - dynamic unrolling through time
# note: terminate session and run again if ValueError arises due to variables
#already in kernel (quickfix)
import numpy as np
import tensorflow as tf

n_inputs = 3
n_neurons = 5

"""
note:
none means dimension is not specified. So, it X0 takes any number of rows but
n_inputs number of columns.
"""
n_steps = 2
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

"""
perm is useful to decide the tranformation to be done.
initial is 0,1,2
transposed is 1,0,2 ie only rows and columns are altered. third dimension is kept same.
it is NOT transposed.
in 2D (0,1) is transposed to (1,0)
in 3D by default (0,1,2) is transposed to (2,1,0)
"""

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

init = tf.global_variables_initializer()

X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
    ])

with tf.Session() as sess:
  init.run()
  outputs_val = outputs.eval(feed_dict={X: X_batch})

  
print(outputs_val)

[[[ 0.9283708   0.7876568   0.5752352  -0.6714994  -0.5580821 ]
  [ 0.9990377   1.         -0.7308792  -0.99996525 -0.9984233 ]]

 [[ 0.9981606   0.99989915  0.18008448 -0.983926   -0.96266776]
  [-0.5025831   0.8806925   0.8061418  -0.94602484  0.08150025]]

 [[ 0.99995434  0.99999994 -0.28321296 -0.9993323  -0.997452  ]
  [ 0.8391648   0.9999999  -0.6400507  -0.9993323  -0.94853944]]

 [[-0.98943305  0.9998604  -0.8851687  -0.19771312 -0.9974578 ]
  [-0.4721411   0.9944253  -0.9710066  -0.8430085   0.44378728]]]


In [1]:
# RNN with dynamic length input time length - dynamic unrolling through time
# note: terminate session and run again if ValueError arises due to variables
#already in kernel (quickfix)
import numpy as np
import tensorflow as tf

n_inputs = 3
n_neurons = 5

"""
note:
none means dimension is not specified. So, it X0 takes any number of rows but
n_inputs number of columns.
"""
n_steps = 2
seq_length = tf.placeholder(tf.int32,[None])
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

"""
perm is useful to decide the tranformation to be done.
initial is 0,1,2
transposed is 1,0,2 ie only rows and columns are altered. third dimension is kept same.
it is NOT transposed.
in 2D (0,1) is transposed to (1,0)
in 3D by default (0,1,2) is transposed to (2,1,0)
"""

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,sequence_length=seq_length)

init = tf.global_variables_initializer()

X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
    ])

seq_length_batch = np.array([2,1,2,2])

with tf.Session() as sess:
  init.run()
  outputs_val, states_val = sess.run([outputs,states],feed_dict={X: X_batch, seq_length: seq_length_batch})

  
print(outputs_val)

[[[ 0.55414253 -0.4947694  -0.7171284   0.9170067   0.25633723]
  [ 0.9999957  -0.99971545 -0.9999953   0.9999999   0.99998003]]

 [[ 0.9974754  -0.9826348  -0.9979176   0.99990296  0.98509854]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.99998885 -0.9995462  -0.9999868   1.          0.9998097 ]
  [ 0.99194723 -0.9195913  -0.9985769   0.9999679   0.99783736]]

 [[ 0.99169165 -0.9995949  -0.96634746  0.95965934  0.99997634]
  [ 0.06266692  0.24095273 -0.79903674  0.97291553  0.8515346 ]]]
